# Collaborative Filtering from Scratch

### **Table of Content :**
1. **[Load the data](#section1)**


2. **[Preprocessing](#section2)**
    * 2-1. Subsampling
    * 2-2. Assign new Id values to Users and Movies
    * 2-3. Split the data into train & test set
    * 2-4. Creat looking-up dictionaries for users, movies and ratings
  

3. **[User-User Collaborative Filtering](#section3)**
    * 3-1. Count the number of users and movies
    * 3-2. Get the averages, deviations and the weights
    * 3-3. Predict the ratings
    
  
4. **[Item-Item Collaborative Filtering](#section4)**
    * 4-1. Get the averages, deviations and the weights
    * 4-3. Predict the ratings

In [1]:
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.utils import shuffle
from sortedcontainers import SortedList

## 1. Load the data <a id = 'section1'></a>

In [2]:
dir = '../'
rt = pd.read_csv(dir + 'data/rating.csv')
mo = pd.read_csv(dir + 'data/movie.csv')

# Joing the two data frame
df = pd.merge(rt, mo, how = 'inner', on = ['movieId'])
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,1996-12-25 15:26:09,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,1996-11-27 08:19:02,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,1996-06-23 20:36:14,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,1996-10-28 13:29:44,Jumanji (1995),Adventure|Children|Fantasy


In [3]:
df = df.drop(columns = ['timestamp', 'title', 'genres'])
df.head()

,userId,movieId,rating
0,1,2,3.5
1,5,2,3.0
2,13,2,3.0
3,29,2,3.0
4,34,2,3.0


## 2. Preprocessing <a id = 'section2'></a>

### 2-1. Sub-sampling 

In [4]:
# Make the user Id starts from 0 
df.userId -= 1
df.head()

,userId,movieId,rating
0,0,2,3.5
1,4,2,3.0
2,12,2,3.0
3,28,2,3.0
4,33,2,3.0


In [5]:
print("The number of samples of the data is ", len(df))

The number of samples of the data is  20000263


Let's extract only the useful subset data whose users and movies have many ranting data. 

In [6]:
N = df.userId.max() + 1
M = df.movieId.nunique() + 1

print("The number of Users is ", N)
print("The number of Movies is ", M)

The number of Users is  138493
The number of Movies is  26745


In [7]:
user_ids_count = Counter(df.userId)
movie_ids_count = Counter(df.movieId)

The outcome of `Counter()` will be **'column value : count_number'**. So take only the column values from the most common ones.

In [8]:
# Choose the numbers to subset 
n = 1000
m = 200 

user_ids = [col for col, idx in user_ids_count.most_common(n)]
movie_ids = [col for col, idx in movie_ids_count.most_common(m)]

In [9]:
# Filter the data only with the user id and movie id mostly rated
df_sub = df[df.userId.isin(user_ids) & df.movieId.isin(movie_ids)]

In [10]:
df_sub.head()

,userId,movieId,rating
20,155,2,5.0
85,571,2,3.5
89,585,2,3.0
123,740,2,3.0
129,767,2,3.0


### 2-2. Assign new Id values to Users and Movies

Now there are sparsity in the values of `userId` and `movieID`. Create the new ids for users and movies in the `df_sub` DataFrame.   

In [11]:
user_dic = {}
i = 0

for k in user_ids:
    user_dic[k] = i
    i += 1

In [12]:
movie_dic = {}
i = 0

for k in movie_ids:
    movie_dic[k] = i
    i += 1

In [13]:
df_sub['user_idx'] = df_sub.userId.apply(lambda x: user_dic[x])
df_sub['movie_idx'] = df_sub.movieId.apply(lambda x: movie_dic[x])

df_sub = df_sub.reset_index(drop = True)
df_sub.head()

C:\Users\jjone\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\jjone\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userId,movieId,rating,user_idx,movie_idx
0,155,2,5.0,190,125
1,571,2,3.5,849,125
2,585,2,3.0,661,125
3,740,2,3.0,176,125
4,767,2,3.0,348,125


### 2-3. Split the data into train & test set

In [14]:
# Shuffle the data
df_sub = shuffle(df_sub)

In [15]:
# split the data
cut = int(0.8*len(df_sub))

tr = df_sub.iloc[:cut]
te = df_sub.iloc[cut:]

In [16]:
# Reset the index for each dataset 
tr = tr.reset_index(drop = True)
te = te.reset_index(drop = True)

In [17]:
print("The size of train : ", len(tr))
print("The size of test : ", len(te))

The size of train :  133628
The size of test :  33407


### 2-4. Creat looking-up dictionaries for users, movies and ratings

Create dictionaries for eaily looking up user_ids, movies_ids and ratings. 

|        Name       |               Forms              | 
|-------------------|:--------------------------------:|
| **user_to_movie** |       *{user_id : movie_id}*     |
| **movie_to_user** |       *{movie_id : user_id}*     |
| **um_to_rating**  | *{(user_id, movie_id) : rating}* |
    

In [20]:
user_to_movie = {}
movie_to_user = {}
um_to_rating_tr = {}
count = 0

# Create user, movie look-up dictionary with train set
def making_user_movie_dics_train(row):

    global count

    a = int(row.user_idx)    # The user id of the given row
    m = int(row.movie_idx)   # Tue movie id of the given row
    r = row.rating           # The rating that the user gave to the movie

    # Fill the "user to movie" dict
    if a not in user_to_movie:
        user_to_movie[a] = [m]
    else:
        user_to_movie[a].append(m)

    # Fill the "movie to user" dict
    if m not in movie_to_user:
        movie_to_user[m] = [a]
    else:
        movie_to_user[m].append(a)

    # Fill the "(user, movie) to rating" dict
    um_to_rating_tr[(a, m)] = r
    
    count += 1
    if count % 10000 == 0:
        print("==========Processed: %.2f" % (count/len(tr)))

In [21]:
# Apply the function
_ = tr.apply(making_user_movie_dics_train, axis = 1)

==========Processed: 0.07
==========Processed: 0.15
==========Processed: 0.22
==========Processed: 0.30
==========Processed: 0.37
==========Processed: 0.45
==========Processed: 0.52
==========Processed: 0.60
==========Processed: 0.67
==========Processed: 0.75
==========Processed: 0.82
==========Processed: 0.90
==========Processed: 0.97


As test set has no ratings (we're predicting it), it has to be processed without rating. 

In [22]:
# Create user, movie look-up dictionary with test set
um_to_rating_te = {}
count = 0

def making_user_movie_dics_test(x):

    global count

    a = int(x.user_idx)
    m = int(x.movie_idx)
    r = x.rating

    # Fill the "(user, movie) to rating" dict
    um_to_rating_te[(a, m)] = r
        
    count += 1
    if count % 10000 == 0:
        print("==========Processed: %.2f" % (count/len(te)))

In [23]:
_ = te.apply(making_user_movie_dics_test, axis = 1)

==========Processed: 0.30
==========Processed: 0.60
==========Processed: 0.90


## 3. User-User Collaborative Filtering <a id = 'section3'></a>

User-User collaborative filtering is getting the similarites of users and provide the recommendation with the movies that the users with similar taste liked. 

### $Score(a, m) = \bar{r}$<sub>a</sub> + $\sum_{b=1}^{N} W$<sub>a_b</sub> * ($r$<sub>b_m</sub> - $\bar{r}$<sub>b</sub>) $/$ $\sum_{b=1}^{N} |W$<sub>a_b</sub>$|$ 

- $\bar{r}$<sub>a</sub> : the average rating of user a 
- W<sub>a_b</sub> : the similarities between user a & b
- ($r$<sub>b_m</sub> - $\bar{r}$<sub>b</sub>) : the rating deviation of the user b to the movie m 

### 3-1. Count the number of users and movies 

In [24]:
# Count the number of users
N = np.max(list(user_to_movie.keys())) + 1                               # User id starts from 0

In [25]:
# Count the number of movies
m_tr = np.max(list(movie_to_user.keys()))

In [26]:
# Get the maximum movie id from the test set for the movies not in the train set
te_movie_list = [m for (u, m), r in um_to_rating_te.items()]
m_te = np.max(te_movie_list)

In [27]:
# Total Number of movies both from train & test
M = max(m_tr, m_te) + 1

print("The size of the data: {} users & {} movies".format(N, M))

The size of the data: 1000 users & 200 movies


### 3-2. Get the averages, deviations and the weights

Now get the average and deviation of ratings for each user and the similarities between users. Terms to be calculated as follows:

|        Name       |               Values              |      Final Outcome   |
|:-------------------:|:--------------------------------:|-------------------------------:|
|      **r_avg**    |the average rating for each user| the list of averages (**averages**) |
|    **r_dev_dic**  |the deviation of rating for each user and movie| the list of deviations (**deviations**)  |
|      **w_ab**     | the similarities between user a and b| the list of neighbors with the weights (**neigbors**)  |

In [28]:
K = 25                  # the number of neights to consider
similarity_limit = 5    # the minimum number of movies users must have in common

In [29]:
averages = []           # ----> the list of averages
deviations = []         # ----> the list of deviations
neighbors = []          # ----> the list of neighbors with the weights

for a in range(N):
    
    # Get the movie lists rated by user a
    movies_a = user_to_movie[a]
    
    # Create the set of the movies to get the common movie list with user b
    movies_a_set = set(movies_a)

    # Create the "movie : rating" dictionary and get the rating average by user a
    r_dic_a = { m : um_to_rating_tr[(a, m)] for m in movies_a }
    r_avg_a = np.mean(list(r_dic_a.values()))                           # average rating of user a

    # Create the "movie : rating deviation" dictionary and get the deviation by user a
    r_dev_dic_a = { m : (r - r_avg_a) for m, r in r_dic_a.items() }
    r_dev_arr_a = np.array(list(r_dev_dic_a.values()))
    r_sigma_a = np.sqrt(np.dot(r_dev_arr_a, r_dev_arr_a))                # for calculating the user similarities

    # Save the average and deviation value
    averages.append(r_avg_a)
    deviations.append(r_dev_arr_a)

    w_neighbor = SortedList()
    for b in range(N):
        if b != a:
            
            # Get the movie lists rated by user b
            movies_b = user_to_movie[b]

            # Create the set of the movies to get the common movie list with user b
            movies_b_set = set(movies_b)
            movies_a_b = (movies_a_set & movies_b_set)                   # common movies by user a & b (Intersection)

            # Get the rating average and deviation by the other users whose similarities are above the limit
            if len(movies_a_b) > similarity_limit:

                # Create the "movie : rating" dictionary and get the rating average by user b
                r_dic_b = { m : um_to_rating_tr[(b, m)] for m in movies_b }
                r_avg_b = np.mean(list(r_dic_b.values()))                    # average rating of user b

                # Create the "movie : rating deviation" dictionary and get the deviation by user b
                r_dev_dic_b = { m : (r - r_avg_b) for m, r in r_dic_b.items() }
                r_dev_arr_b = np.array(list(r_dev_dic_b.values()))
                r_sigma_b = np.sqrt(np.dot(r_dev_arr_b, r_dev_arr_b))    # for calculating the user similarities

                # User similarities ( <- correlation weights)
                numerator = sum(r_dev_dic_a[m] * r_dev_dic_b[m] for m in movies_a_b)
                w_ab = numerator / (r_sigma_a * r_sigma_b)

                # Add to the sorted list (negative weights for descending)
                w_neighbor.add((-w_ab, b))

                # If the number of neighbors is obove the limit K, delete the last one
                if len(w_neighbor) > K:
                    del w_neighbor[-1]

    # Save the neightbor
    neighbors.append(w_neighbor)

    if a % 100 == 0:
        print(a)            # tracking the preprocessing

0
100
200
300
400
500
600
700
800
900


### 3-3. Predict the ratings

In [41]:
# Create the scoring function 
def predict_scoring(a, m):
    '''
    Predicting the rating of user a to movie b
    input : user_id (a) and movie_id (m)
    output : the predicted rating
    '''
    # Initialization
    numerator = 0
    denominator = 0
    for w_ab, b in neighbors[a]:
        try:
            w_ab *= -1
            numerator += w_ab * deviations[b][m]
            denominator += abs(w_ab)

        # For the case that the neighbors of user a didn't rate the given movie m
        except:
            pass

    # If the sum of weights is 0 (the sum of the similarities values = 0)
    if denominator == 0:
        pred = averages[a]
    else:
        pred = averages[a] + numerator / denominator

    # The fixed range of rating is between .5 and 5
    pred = min(5, pred)
    pred = max(.5, pred)

    return pred

In [35]:
# Create the get_loss function 
def get_loss(pred, actual):
    '''
    Calculating the loss by MSE
    input: the list of the prediction and actual values
    output : mean_squared_error
    '''
    loss = np.array(pred) - np.array(actual)
    return np.mean(loss**2)

In [42]:
# Get the prediction and actual value from train set
tr_preds = []
tr_actual = []

for (a, m), r in um_to_rating_tr.items():
    # Predict the score of movie m by user a
    pred = predict_scoring(a, m)

    tr_preds.append(pred)
    tr_actual.append(r)

In [43]:
# Get the prediction and actual value from test set
te_preds = []
te_actual = []

for (a, m), r in um_to_rating_te.items():
    # Predict the score of movie m by user a
    pred = predict_scoring(a, m)

    te_preds.append(pred)
    te_actual.append(r)

In [47]:
# Get the loss
tr_loss = get_loss(tr_preds, tr_actual)
te_loss = get_loss(te_preds, te_actual)

print("The loss of train set : %.4f" % tr_loss)
print("The loss of test set :  %.4f" % te_loss)

The loss of train set : 0.8552
The loss of test set :  0.8746


## 4. Item-Item Collaborative Filtering <a id = 'section4'></a>

Item-Item collaborative filtering is getting the similarites of items and provide the recommendation with the simiiar items that the given user would like. This can be thought in an opposite way of User-User collaborative filtering and the whole process is the same. Flip the original *User_to_Movie* matrix to *Movie_to_User* matrix. Imagine the items give ratings to users and **find the users the items would prefer**.

The pratical difference is the number of data for computing. When comparing between items, there are much more data than comparing between users (up to ~ 100k users **vs.** up to ~ 20k items to look up). Therefore, the item-based CF can have the weighted values based on more data. 

Plus, calculating scores require $M^2*N$ times while for user-based CF, $N^2*M$. Considering N >> M, it's easy to understand that the item-based CF is faster than user-based CF. 

### 4-1. Get the averages, deviations and the weights



|        Name       |               Values              |      Final Outcome   |
|-------------------|--------------------------------|-------------------------------:|
|      **r_avg**    |the average rating for each user| the list of averages (**averages**) |
|    **r_dev_dic**  |the deviation of rating for each user and movie| the list of deviations (**deviations**)  |
|      **w_mn**     | the similarities between movie a and b| the list of neighbors with the weights (**neigbors**)  |

In [48]:
K = 20                  # the number of neights to consider
similarity_limit = 5    # the minimum number of movies users must have in common

In [50]:
averages = []           # ----> the list of averages
deviations = []         # ----> the list of deviations
neighbors = []          # ----> the list of neighbors with the weights

for m in range(M):

    # Get the user lists who rated movie m 
    users_m = movie_to_user[m]

    # Create the set of the users to get the common user list with movie n 
    users_m_set = set(users_m)

    # Create the "user : rating" dictionary and get the rating average to the movie m 
    r_dic_m = { a : um_to_rating_tr[(a, m)] for a in users_m }
    r_avg_m = np.mean(list(r_dic_m.values()))                                   # average rating of the movie m

    # Create the "user : rating deviation" dictionary and get the deviation to the movie m
    r_dev_dic_m = { a : (r - r_avg_m) for a, r in r_dic_m.items() }             # how the user ratings deviate about the movie m 
    r_dev_arr_m = np.array(list(r_dev_dic_m.values()))
    r_sigma_m = np.sqrt(np.dot(r_dev_arr_m, r_dev_arr_m))                       # for calculating the movie similarities

    # Save the average and deviation value 
    averages.append(r_avg_m)
    deviations.append(r_dev_arr_m)

    w_neighbor = SortedList()
    for n in range(M):
        if n != m:

            # Get the user lists rated by movie n
            users_n = movie_to_user[n]

            # Create the set of the users to get the common user list with movie n
            users_n_set = set(users_n)
            users_m_n = (users_m_set & users_n_set)                             # common users by movie m & n (Intersection)

            # Get the rating average and deviation of the other movies whose similarities are above the limit
            if len(users_m_n) > similarity_limit:

                # Create the "user : rating" dictionary and get the rating average by movie n
                r_dic_n = { a : um_to_rating_tr[(a, n)] for a in users_n }
                r_avg_n = np.mean(list(r_dic_n.values()))                       # average rating to the movie n

                # Create the "user : rating deviation" dictionary and get the deviation to the movie n
                r_dev_dic_n = { a : (r - r_avg_n) for a, r in r_dic_n.items() }
                r_dev_arr_n = np.array(list(r_dev_dic_n.values()))
                r_sigma_n = np.sqrt(np.dot(r_dev_arr_n, r_dev_arr_n))           # for calculating the movie similarities

                # movie similarities (Pearson correlation)
                numerator = sum(r_dev_dic_m[a] * r_dev_dic_n[a] for a in users_m_n)
                w_mn = numerator / (r_sigma_m * r_sigma_n)

                # Add to the sorted list (negative weights for descending)
                w_neighbor.add((-w_mn, n))

                # If the number of neighbors is obove the limit K, delete the last one
                if len(w_neighbor) > K:
                    del w_neighbor[-1]

    # Save the neightbor
    neighbors.append(w_neighbor)

    if a % 100 == 0:
        print("==========Processed: {}%".format(a//N))            # tracking the preprocessing

### 4-2. Predict the ratings

In [51]:
def predict_scoring(a, m):
    '''
    Predicting the rating of user a to movie b
    input : user_id (a) and movie_id (m)
    output : the predicted rating
    '''
    # Initialization
    numerator = 0
    denominator = 0
    
    for w_mn, n in neighbors[m]:
        try:
            w_mn *= -1
            numerator += w_mn * deviations[n][a]
            denominator += abs(w_mn)

        # For the case that the neighbors of the given movie n hasn't rate the given user a
        except:
            pass

    # If the sum of weights is 0 (the sum of the similarities values = 0)
    if denominator == 0:
        pred = averages[m]
    else:
        pred = averages[m] + numerator / denominator

    # The fixed range of rating is between .5 and 5
    pred = min(5, pred)
    pred = max(.5, pred)

    return pred

In [52]:
def get_loss(pred, actual):
    '''
    Calculating the loss by MSE
    input: the list of the prediction and actual values
    output : mean_squared_error
    '''
    loss = np.array(pred) - np.array(actual)
    return np.mean(loss**2)

In [53]:
# Get the prediction and actual value from train set
tr_preds = []
tr_actual = []

for (a, m), r in um_to_rating_tr.items():
    # Predict the score of movie m by user a
    pred = predict_scoring(a, m)

    tr_preds.append(pred)
    tr_actual.append(r)

In [54]:
# Get the prediction and actual value from test set
te_preds = []
te_actual = []

for (a, m), r in um_to_rating_te.items():
    # Predict the score of movie m by user a
    pred = predict_scoring(a, m)

    te_preds.append(pred)
    te_actual.append(r)

In [55]:
# Get the loss
tr_loss = get_loss(tr_preds, tr_actual)
tr_loss = get_loss(te_preds, te_actual)

In [56]:
print("The loss of train set : %.4f" % tr_loss)
print("The loss of test set :  %.4f" % te_loss)

The loss of train set : 0.8567
The loss of test set :  0.8746
